# Generowanie ciągu S (bits)

In [1]:
import sympy

primes = sympy.primerange(1_000, 10_000)

In [2]:
congruent_primes = [prime for prime in primes if prime % 4 == 3]

In [3]:
import random

p, q = random.sample(congruent_primes, 2)
p, q

(6863, 5651)

In [4]:
N = p * q
N

38782813

In [5]:
import math

# szukaj liczby relatywnie pierwszej tak długo aż się uda.
# ponieważ p * q daje ogromną liczbę przy założeniu, że p i q są liczbami 4-cyfrowymi
# to znalezienie takiej liczby powinno być bardzo łatwe i szybkie
def next_random_coprime_of(n):
    if n == 0 or n == 1:
        raise ValueError("Cannot find coprime of 0 or 1.")
    while True:
        random_coprime = random.randint(1, n - 1)
        while math.gcd(random_coprime, n) != 1 and random_coprime < n:
            random_coprime += 1
        if random_coprime != n:
            return random_coprime

In [6]:
x = next_random_coprime_of(N)
x

2821980

In [7]:
x0 = pow(x, 2, N)
x0

24647419

In [8]:
bits = []
xi = x0
for i in range(20_000):
    xi = pow(xi, 2, N)
    bits.append(xi & 1)

# Testy

In [9]:
def test_result(name, passed):
    print(f"{name}: {'ZDANY' if passed else 'NIEZDANY'}")

## Test pojedynczych bitów

In [10]:
from collections import Counter

counts = Counter(bits)
test1_passed = 9725 < counts[1] and counts[1] < 10275
test_result("Test pojedynczych bitów", test1_passed)

Test pojedynczych bitów: ZDANY


## Test serii

In [11]:
def calc_sequences(bits):
    seqs_of_zero = {}
    seqs_of_one = {}
    seqs_of_zero_capped = {}
    seqs_of_one_capped = {}

    i = 0
    sl = 1
    sb = -1
    while i < len(bits):
        if bits[i] == sb:
            sl += 1
        else:
            if sb == 0:
                if not sl in seqs_of_zero:
                    seqs_of_zero[sl] = 0
                seqs_of_zero[sl] += 1
                if sl >= 6:
                    sl = 6
                if not sl in seqs_of_zero_capped:
                    seqs_of_zero_capped[sl] = 0
                seqs_of_zero_capped[sl] += 1
            elif sb == 1:
                if not sl in seqs_of_one:
                    seqs_of_one[sl] = 0
                seqs_of_one[sl] += 1
                if sl >= 6:
                    sl = 6
                if not sl in seqs_of_one_capped:
                    seqs_of_one_capped[sl] = 0
                seqs_of_one_capped[sl] += 1
            sb = bits[i]
            sl = 1
        i += 1
    return Counter(seqs_of_zero), Counter(seqs_of_one), Counter(seqs_of_zero_capped), Counter(seqs_of_one_capped)

In [12]:
seq_counts_zero, seq_counts_one, seq_counts_zero_capped, _ = calc_sequences(bits)
print(seq_counts_zero_capped)
test2_passed = sum([
    2315 <= seq_counts_zero_capped[1] and seq_counts_zero_capped[1] <= 2685,
    1114 <= seq_counts_zero_capped[2] and seq_counts_zero_capped[2] <= 1386,
    527 <= seq_counts_zero_capped[3] and seq_counts_zero_capped[3] <= 723,
    240 <= seq_counts_zero_capped[4] and seq_counts_zero_capped[4] <= 384,
    103 <= seq_counts_zero_capped[5] and seq_counts_zero_capped[5] <= 209,
    103 <= seq_counts_zero_capped[6] and seq_counts_zero_capped[6] <= 209
]) == 6
test_result("Test serii", test2_passed)

Counter({1: 2450, 2: 1282, 3: 632, 4: 326, 6: 170, 5: 134})
Test serii: ZDANY


## Test długiej serii

In [13]:
test3_passed = len([c for c in seq_counts_zero if c >= 26]) == 0 and len([c for c in seq_counts_one if c >= 26]) == 0
test_result("Test długiej serii", test3_passed)

Test długiej serii: ZDANY


## Test pokerowy

In [14]:
segments_counts = Counter()

for i in range(0, len(bits), 4):
    four_bits = bits[i:i+4]
    num = ''.join(map(str, four_bits))
    segments_counts[num] += 1

X = (16/5000) * sum([i * i for i in segments_counts.values()]) - 5000

print(f"X = {X}")
test4_passed = 2.16 <= X and X <= 46.17
test_result("Test pokerowy", test4_passed)

X = 27.558399999999892
Test pokerowy: ZDANY


## Wyniki testów

In [15]:
test_result("Test pojedynczych bitów", test1_passed)
test_result("Test serii", test2_passed)
test_result("Test długiej serii", test3_passed)
test_result("Test pokerowy", test4_passed)

Test pojedynczych bitów: ZDANY
Test serii: ZDANY
Test długiej serii: ZDANY
Test pokerowy: ZDANY
